### Important
The taboo_data folder is not included, but is needed for creating the human and original data csv files. For where to download, please check the root README.md

In [1]:
### Important
The taboo_data folder is not included, but is needed for creating the human and original data csv files. For where to download, please check the root README.mdimport os
import openai

import pandas as pd
import os
import numpy as np
from ast import literal_eval
import json
import pickle
import time

In [6]:
default_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}"'

In [7]:
openai.api_key = # removed, paste your own here

In [8]:
def request_response_from_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        #prompt=prompt,
        messages=[
            {"role": "system", "content": "You are a crowdsourcing worker that earns a living through creating paraphrases."},
            {"role": "user", "content": prompt}],
        temperature=1,
        frequency_penalty=0.0,
        presence_penalty=1.5,
        n=2)
    return response

In [9]:
def request_with_checks(prompt):
    success = False
    count = 0
    while not success:
        if count > 0:
            print(f'Retrying with again. Current number of retries: {count}')
        if count >= 5:
            raise Exception('Too many attempts')
        try:
            response = request_response_from_gpt(prompt)
            sucess = True
            break
        except openai.error.RateLimitError as e:
            print(e)
            time.sleep(10)
            count += 1
        except openai.error.APIConnectionError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.APIError or openai.error.JSONDecodeError:
            time.sleep(5)
            count += 1
    return response

# LIU

In [10]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/liu/config_0.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [11]:
dct_phrases = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']

In [12]:
dct_phrases.keys()

dict_keys(['cooking_recipe', 'datetime_query', 'audio_volume_up', 'news_query', 'audio_volume_down', 'weather_query', 'qa_currency', 'play_music', 'transport_traffic', 'music_query'])

In [13]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(default_prompt.format(phrase))

In [14]:
dct_responses = {}

In [14]:
# collect data

for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "show me the recipe for cookies"
Rephrase an original question or statement 3 times. Original phrase: "at what temperature should i bake cookies"
Rephrase an original question or statement 3 times. Original phrase: "show me a cookie recipe"
Rephrase an original question or statement 3 times. Original phrase: "find the correct temperature for frying a whole turkey"
Rephrase an original question or statement 3 times. Original phrase: "what is the best meatball recipe"
Rephrase an original question or statement 3 times. Original phrase: "what is the best oven temperature to roast potatoes"
Rephrase an original question or statement 3 times. Original phrase: "look up recipes for four cheese omelette"
Rephrase an original question or statement 3 times. Original phrase: "hey robot give me a list of ingredients for welsh rarebit"
Rephrase an original question or statement 3 times. Original phrase: "i want to cook that meal

Rephrase an original question or statement 3 times. Original phrase: "play music by the beatles"
Rephrase an original question or statement 3 times. Original phrase: "start playlist from track one"
Rephrase an original question or statement 3 times. Original phrase: "play funk music"
Rephrase an original question or statement 3 times. Original phrase: "play newest bob dylan playlist"
Rephrase an original question or statement 3 times. Original phrase: "play let me love you by ariana grande"
Rephrase an original question or statement 3 times. Original phrase: "please play music from my playlist"
That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 14bee9cb1f8288d41c0e68c89aac6220 in your message.)
Retrying with again. Current number of retries: 1
Rephrase an original question or statement 3 times. Original phrase: "play artist"
8
Rephrase an origi

In [15]:
with open('challenge_data/liu/liu_0.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [16]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [17]:
fb_0 = pd.DataFrame.from_dict(dct_df)

In [18]:
fb_0.to_csv('challenge_data/liu/liu_0.csv', index=False)

### ADD TWO TABOO WORDS

In [56]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/liu/config_2.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [26]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}” or “{}” in your responses.'

In [27]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1]))

In [28]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "show me the recipe for cookies". Don’t use the words “recipe” or “recipes” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "at what temperature should i bake cookies". Don’t use the words “recipe” or “recipes” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me a cookie recipe". Don’t use the words “recipe” or “recipes” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find the correct temperature for frying a whole turkey". Don’t use the words “recipe” or “recipes” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the best meatball recipe". Don’t use the words “recipe” or “recipes” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the best oven temperature to roast potatoes". Don’t use the words “rec

Rephrase an original question or statement 3 times. Original phrase: "please talk softer". Don’t use the words “quieter” or “lower” in your responses.
5
Rephrase an original question or statement 3 times. Original phrase: "is it raining in new brighton pa". Don’t use the words “weather” or “temperature” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is it suppose to rain today". Don’t use the words “weather” or “temperature” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "will i need my sun glasses today". Don’t use the words “weather” or “temperature” in your responses.
That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 37efd1d034d38ad47ee109823551ea0e in your message.)
Retrying with again. Current number of retries: 1
Rephrase an original question or statement 

Rephrase an original question or statement 3 times. Original phrase: "tell me my music likes". Don’t use the words “singer” or “song” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "name the singer". Don’t use the words “singer” or “song” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what year did this song come out". Don’t use the words “singer” or “song” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find me the song currently playing". Don’t use the words “singer” or “song” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the track information currently". Don’t use the words “singer” or “song” in your responses.


In [29]:
with open('challenge_data/liu/liu_2.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [30]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_2 = pd.DataFrame.from_dict(dct_df)
fb_2.to_csv('challenge_data/liu/liu_2.csv', index=False)

In [57]:
# WE CAN FILTER FOR TABOO WORDS
fb_2 = pd.read_csv('challenge_data/liu/liu_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_2['no_taboo_words'] = fb_2.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_2 = fb_2[fb_2['no_taboo_words'] == True]

#2349 - 2239 after filtering
fb_2.to_csv('challenge_data/liu/liu_2_filt.csv', index=False)

### ADD FOUR TABOO WORDS

In [59]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/liu/config_4.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [33]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}” or “{}” in your responses.'

In [34]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3]))

In [35]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "show me the recipe for cookies". Don’t use the words “recipe”, “recipes”, “cooking” or “cook” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "at what temperature should i bake cookies". Don’t use the words “recipe”, “recipes”, “cooking” or “cook” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me a cookie recipe". Don’t use the words “recipe”, “recipes”, “cooking” or “cook” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find the correct temperature for frying a whole turkey". Don’t use the words “recipe”, “recipes”, “cooking” or “cook” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the best meatball recipe". Don’t use the words “recipe”, “recipes”, “cooking” or “cook” in your responses.
Rephrase an original question or statement 3 times. O

Rephrase an original question or statement 3 times. Original phrase: "speak quieter". Don’t use the words “quieter”, “lower”, “decrease” or “down” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "low the volume". Don’t use the words “quieter”, “lower”, “decrease” or “down” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "turn down media volume". Don’t use the words “quieter”, “lower”, “decrease” or “down” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "lower volume to half". Don’t use the words “quieter”, “lower”, “decrease” or “down” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "please talk softer". Don’t use the words “quieter”, “lower”, “decrease” or “down” in your responses.
5
Rephrase an original question or statement 3 times. Original phrase: "is it raining in new brighton pa". Don’t use the words “weather”, “temperatu

9
Rephrase an original question or statement 3 times. Original phrase: "find me the song currently playing". Don’t use the words “singer”, “song”, “listening” or “music” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what music genre do i listen to the most". Don’t use the words “singer”, “song”, “listening” or “music” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "who's that song by". Don’t use the words “singer”, “song”, “listening” or “music” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "email me the lyrics to this song". Don’t use the words “singer”, “song”, “listening” or “music” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "give me information on the song that 's playing". Don’t use the words “singer”, “song”, “listening” or “music” in your responses.
Rephrase an original question or statement 3 times. Original p

In [36]:
with open('challenge_data/liu/liu_4.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [37]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_4 = pd.DataFrame.from_dict(dct_df)
fb_4.to_csv('challenge_data/liu/liu_4.csv', index=False)

In [61]:
# WE CAN FILTER FOR TABOO WORDS
fb_4 = pd.read_csv('challenge_data/liu/liu_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_4['no_taboo_words'] = fb_4.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_4 = fb_4[fb_4['no_taboo_words'] == True]

fb_4.to_csv('challenge_data/liu/liu_4_filt.csv', index=False)

### ADD SIX FINAL TABOO WORDS

In [63]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/liu/config_6.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [16]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}”, “{}”, “{}” or “{}” in your responses.'

In [17]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3], dct_taboo[key][4], dct_taboo[key][5]))

In [18]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "show me the recipe for cookies". Don’t use the words “recipe”, “recipes”, “cooking”, “cook”, “ingredients” or “substitute” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "at what temperature should i bake cookies". Don’t use the words “recipe”, “recipes”, “cooking”, “cook”, “ingredients” or “substitute” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me a cookie recipe". Don’t use the words “recipe”, “recipes”, “cooking”, “cook”, “ingredients” or “substitute” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find the correct temperature for frying a whole turkey". Don’t use the words “recipe”, “recipes”, “cooking”, “cook”, “ingredients” or “substitute” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the best meatball recipe". Don’t use the wor

Retrying with again. Current number of retries: 1
Rephrase an original question or statement 3 times. Original phrase: "what 's going on at the capitol". Don’t use the words “news”, “headlines”, “trending”, “happened”, “headline” or “updates” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the news this morning". Don’t use the words “news”, “headlines”, “trending”, “happened”, “headline” or “updates” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what 's the latest on trump 's travel ban". Don’t use the words “news”, “headlines”, “trending”, “happened”, “headline” or “updates” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "anything new on facebook". Don’t use the words “news”, “headlines”, “trending”, “happened”, “headline” or “updates” in your responses.
4
Rephrase an original question or statement 3 times. Original phrase: "lower the volume please". Don

Rephrase an original question or statement 3 times. Original phrase: "play music by the beatles". Don’t use the words “queue”, “listen”, “hear”, “play”, “selected” or “playlist” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "start playlist from track one". Don’t use the words “queue”, “listen”, “hear”, “play”, “selected” or “playlist” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "play funk music". Don’t use the words “queue”, “listen”, “hear”, “play”, “selected” or “playlist” in your responses.
Retrying with again. Current number of retries: 1
Rephrase an original question or statement 3 times. Original phrase: "play newest bob dylan playlist". Don’t use the words “queue”, “listen”, “hear”, “play”, “selected” or “playlist” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "play let me love you by ariana grande". Don’t use the words “queue”, “listen”, “hear”, “play”

In [19]:
with open('challenge_data/liu/liu_6.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [20]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_6 = pd.DataFrame.from_dict(dct_df)
fb_6.to_csv('challenge_data/liu/liu_6.csv', index=False)

In [64]:
# WE CAN FILTER FOR TABOO WORDS
fb_6 = pd.read_csv('challenge_data/liu/liu_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_6['no_taboo_words'] = fb_6.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_6 = fb_6[fb_6['no_taboo_words'] == True]

fb_6.to_csv('challenge_data/liu/liu_6_filt.csv', index=False)

### PREPARE DATASET 

In [2]:
fb_0 = pd.read_csv('challenge_data/liu/liu_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/liu/liu_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/liu/liu_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/liu/liu_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

In [66]:
fb_0 = pd.read_csv('challenge_data/liu/liu_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/liu/liu_2_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/liu/liu_4_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/liu/liu_6_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

Proceed with either taboo samples or without them

In [67]:
fb = fb_0.append(fb_2).append(fb_4).append(fb_6).drop_duplicates('text')

In [68]:
dct = {'cooking_recipe': 0,
 'datetime_query': 1,
 'audio_volume_up': 2,
 'news_query': 3,
 'audio_volume_down': 4,
 'weather_query': 5,
 'qa_currency': 6,
 'play_music': 7,
 'transport_traffic': 8,
 'music_query': 9
}

def function(x):
    return dct[x]

fb['label'] = fb['intent'].apply(function)
fb = fb.sample(frac=1).reset_index(drop=True)
fb['text'] = fb['text'].apply(lambda x: x.replace('"',''))

In [69]:
fb[['text', 'label']].to_csv('challenge_data/liu/full_liu_gpt_filt.csv', index=False) # either filt or non filt

Now lets prepare user data and the original data into dataset - for human data we have the same preprocessing and for the  original we remove the seed samples, take only needed intents and prepare csv

To get the data for the original version and human version from the original paper, please see the README.md file at the root

In [6]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/liu/liu_restricted_0_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])
    
fb_h_0 = pd.DataFrame.from_dict(dct_dataframe)

In [9]:
f = open('/taboo_data/challenge_data/intent_classification/liu/liu_restricted_2_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_2 = pd.DataFrame.from_dict(dct_dataframe)

f = open('/taboo_data/challenge_data/intent_classification/liu/liu_restricted_4_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_4 = pd.DataFrame.from_dict(dct_dataframe)


f = open('/taboo_data/challenge_data/intent_classification/liu/liu_restricted_6_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_6 = pd.DataFrame.from_dict(dct_dataframe)

In [10]:
fb_h = fb_h_0.append(fb_h_2).append(fb_h_4).append(fb_h_6).drop_duplicates('text')

In [13]:
fb_h['label'] = fb_h['intent'].apply(function)
fb_h = fb_h.sample(frac=1).reset_index(drop=True)
fb_h['text'] = fb_h['text'].apply(lambda x: x.replace('"',''))

In [14]:
fb_h[['text', 'label']].to_csv('challenge_data/liu/full_liu_human.csv', index=False)

#### NOW ORIGINAL WITHOUT SEEDS

In [16]:
intents = list(set(fb_h['intent']))

In [15]:

f = open('/taboo_data/challenge_data/intent_classification/liu/clf_train.json')

# returns JSON object as 
# a dictionary
data = json.load(f)


# Closing file
f.close()

In [52]:

f = open('/taboo_data/challenge_data/intent_classification/liu/clf_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)


# Closing file
f.close()

In [16]:
all_seed_phrases = []
for key in dct_phrases:
    all_seed_phrases.extend(dct_phrases[key])

In [17]:
fin_dct = {}
for key in data.keys():
    if key not in all_seed_phrases:
        fin_dct[key] = data[key]

In [55]:
dct_dataframe = {'text': [], 'intent': []}

for key in data:
    dct_dataframe['intent'].append(data[key])
    dct_dataframe['text'].append(key)

tmp_orig = pd.DataFrame.from_dict(dct_dataframe)
tmp_orig = tmp_orig[tmp_orig['intent'].isin(intents)]
tmp_orig = tmp_orig.sample(frac=1).reset_index(drop=True)
tmp_orig['label'] = tmp_orig['intent'].apply(function)

tmp_orig[['text', 'label']].to_csv('challenge_data/liu/full_liu_orig_test.csv', index=False)

## LETS Train SVM

In [13]:
full_gpt = pd.read_csv('challenge_data/liu/full_liu_gpt_filt.csv').dropna().reset_index(drop=True) # run with and without filt
full_human = pd.read_csv('challenge_data/liu/full_liu_human.csv').dropna().reset_index(drop=True)
full_orig = pd.read_csv('challenge_data/liu/full_liu_orig_train.csv').dropna().reset_index(drop=True)

full_orig_test = pd.read_csv('challenge_data/liu/full_liu_orig_test.csv').dropna().reset_index(drop=True)

In [14]:
import re
import string

full_gpt['text']=full_gpt['text'].apply(lambda x: x.lower())
full_human['text']=full_human['text'].apply(lambda x: x.lower())
full_orig['text']=full_orig['text'].apply(lambda x: x.lower())
full_orig_test['text']=full_orig_test['text'].apply(lambda x: x.lower())

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_human['text']=full_human['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub(' +',' ',x))
full_human['text']=full_human['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub(' +',' ',x))

In [19]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from joblib import dump, load
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import numpy as np
import itertools
from sklearn import svm
from sklearn.model_selection import train_test_split

gpt_res = []
gpt_f1 = []

human_res = []
human_f1 = []

orig_res = []
orig_f1 = []

for idx in range(0, 10):

    train_gpt, test_gpt = train_test_split(full_gpt, test_size=0.2)
    train_human, test_human = train_test_split(full_human, test_size=0.2)
    
    tmp_train = full_orig.sample(frac=0.4, replace=False, random_state=1)
    tmp_test = full_orig_test.sample(frac=1, replace=False, random_state=1)
    
    pipeline = Pipeline([
            ('bow', CountVectorizer()),  
            ('tfidf', TfidfTransformer()),  
            ('c', svm.SVC(probability=True))
        ])
    fit = pipeline.fit(train_human['text'].str.lower(), train_human['label'])
    
    pred=pipeline.predict(test_gpt['text'].str.lower())
    gpt_res.append(accuracy_score(pred,test_gpt['label']))
    gpt_f1.append(f1_score(pred,test_gpt['label'],  average = 'weighted'))
    
    pred=pipeline.predict(test_human['text'].str.lower())
    human_res.append(accuracy_score(pred,test_human['label']))
    human_f1.append(f1_score(pred,test_human['label'],  average = 'weighted'))
    
    pred=pipeline.predict(tmp_test['text'].str.lower())
    orig_res.append(accuracy_score(pred,tmp_test['label']))
    orig_f1.append(f1_score(pred,tmp_test['label'],  average = 'weighted'))

In [20]:
arr = np.array(gpt_res)

print("ACC GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(gpt_f1)

print("F1 GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr))) 

arr = np.array(human_res)

print("ACC HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(human_f1)

print("F1 HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_res)

print("ACC ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_f1)

print("F1 ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

ACC GPT MEAN: 0.8578366445916114 STD: 0.009479430707984059
F1 GPT MEAN: 0.8578142171044307 STD: 0.009276997842679676
ACC HUMAN SAME MEAN: 0.9565789473684211 STD: 0.010823651473029003
F1 HUMAN SAME MEAN: 0.9566079253931962 STD: 0.010955482362828697
ACC ORIG SAME MEAN: 0.8170193192272309 STD: 0.007131346817399622
F1 ORIG SAME MEAN: 0.8108302326744041 STD: 0.008235663934775308
